Setup: Imports & Data Generation

In [ ]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# Data generation utilities
from easy_boston_data import (
    generate_correlated_boston_like_data,
    generate_easy_boston_like_data
)

# Core solvers
from iterative_solvers import ista, fista, fista_delta
from lbfgs import LBFGSSolver

Preprocessing & Regularization Settings

In [ ]:
# 3.1.1 Generate correlated data & standardize
A, b, x_true = generate_correlated_boston_like_data(m=800, seed=42, noise_std=0.1)
A = (A - A.mean(axis=0)) / A.std(axis=0)
b = b - b.mean()

# 3.1.2 Define regularization scenarios
basic_regs = [
    {'name':'lasso',      'alpha1':1.0,   'alpha2':0.0},
    {'name':'elasticnet', 'alpha1':1.0,   'alpha2':1.0},
    {'name':'lasso',      'alpha1':0.5,   'alpha2':0.0},
    {'name':'elasticnet', 'alpha1':0.2,   'alpha2':0.05},
    {'name':'elasticnet', 'alpha1':1e-4,  'alpha2':0.5}
]

Baseline: ISTA vs FISTA

In [ ]:
from objective_functions import compute_objective
from prox_operators import prox_l1, prox_elastic_net
from iterative_solvers import estimate_lipschitz

records = []
n, m = A.shape[1], A.shape[0]
for reg in basic_regs:
    name, a1, a2 = reg['name'], reg['alpha1'], reg['alpha2']
    # Define smooth part g and its gradient grad_g, and prox_h for h
    if name == 'lasso':
        def g(x):
            r = A @ x - b
            return 0.5 * r.dot(r)
        def grad_g(x):
            return A.T @ (A @ x - b)
        prox_h = lambda v, t: prox_l1(v, t * a1)
        L0 = estimate_lipschitz(A)
        L = L0
    elif name == 'elasticnet':
        def g(x):
            r = A @ x - b
            return 0.5 * r.dot(r) + a2 * x.dot(x)
        def grad_g(x):
            return A.T @ (A @ x - b) + 2 * a2 * x
        prox_h = lambda v, t: prox_elastic_net(v, t, a1, a2)
        L0 = estimate_lipschitz(A)
        L = L0 + 2 * a2
    else:
        continue

    # Initial point
    x0 = np.zeros(n)
    # Run ISTA with history to get iterates
    t_start = time.perf_counter()
    x_i, hist_i = ista(
        x0, g, grad_g, prox_h, L,
        backtracking=False, t_init=None,
        max_iter=500, tol=0.0,
        return_history=True
    )
    t_ista = time.perf_counter() - t_start

    # Compute objective history for ISTA
    obj_i = [compute_objective(xk, A, b, name, a1, a2) for xk in hist_i['x']]

    # Run FISTA
    t_start = time.perf_counter()
    x_f, obj_f = fista(A, b, name, a1, a2, max_iter=500)
    t_fista = time.perf_counter() - t_start

    # Convergence plot
    f_star = obj_f[-1]
    ks_i = np.arange(1, len(obj_i) + 1)
    ks_f = np.arange(1, len(obj_f) + 1)
    plt.figure(figsize=(6,4))
    plt.loglog(ks_i, np.minimum.accumulate(np.array(obj_i) - f_star), '-', label='ISTA')
    plt.loglog(ks_f, np.minimum.accumulate(np.array(obj_f) - f_star), '-', label='FISTA')
    plt.title(f"Convergence: ISTA vs FISTA with {name.title()} (α₁={a1}, α₂={a2})")
    plt.xlabel('Iteration number $k$')
    plt.ylabel('Suboptimality: $f(x) - f^*$')
    plt.grid(True, which='both', ls='--', lw=0.5)
    plt.legend(title='Method')
    plt.tight_layout()
    plt.show()

    # Record summary
    records += [
        {'method':'ISTA',  'regularizer':name, 'alpha1':a1, 'alpha2':a2,
         'iterations':len(obj_i), 'runtime_s':t_ista},
        {'method':'FISTA', 'regularizer':name, 'alpha1':a1, 'alpha2':a2,
         'iterations':len(obj_f), 'runtime_s':t_fista}
    ]

df_basic = pd.DataFrame(records)
print(df_basic)

Advanced: ISTA Stopping & FISTA Adaptive-Restart

In [ ]:
# Advanced experiments with ISTA stopping criterion and FISTA adaptive-restart
adv_results = []
tol = 1e-6

for reg in basic_regs:
    name, a1, a2 = reg['name'], reg['alpha1'], reg['alpha2']
    if name == 'ridge':
        continue

    # Define g, grad_g, prox_h, L as in Section 3.2
    if name == 'lasso':
        def g(x): return 0.5 * np.linalg.norm(A @ x - b)**2
        def grad_g(x): return A.T @ (A @ x - b)
        prox_h = lambda v, t: prox_l1(v, t * a1)
        L0 = estimate_lipschitz(A)
        L = L0
    else:  # elasticnet
        def g(x): return 0.5 * np.linalg.norm(A @ x - b)**2 + a2 * x.dot(x)
        def grad_g(x): return A.T @ (A @ x - b) + 2 * a2 * x
        prox_h = lambda v, t: prox_elastic_net(v, t, a1, a2)
        L0 = estimate_lipschitz(A)
        L = L0 + 2 * a2

    # Initial guess
    x0 = np.zeros(n)

    # ISTA with stopping (backtracking can be enabled if desired)
    t0 = time.perf_counter()
    x_i2, hist_i2 = ista(
        x0, g, grad_g, prox_h, L,
        backtracking=False,
        t_init=None,
        max_iter=500,
        tol=tol,
        return_history=True
    )
    t_ista2 = time.perf_counter() - t0
    step_i2 = hist_i2['step_sizes']
    xhist_i2 = hist_i2['x']

    # FISTA with adaptive restart and stopping
    t0 = time.perf_counter()
    x_f2, obj_f2, step_f2, ratio_f2, xhist_f2 = fista(
        A, b, name, a1, a2,
        max_iter=500,
        tol=tol,
        adaptive_restart=True,
        restart_threshold=1.0,
        return_history=True
    )
    t_fista2 = time.perf_counter() - t0

    adv_results += [
        {
            'method': 'ISTA+stop',
            'regularizer': name,
            'time_s': t_ista2,
            'step_norms': step_i2,
            'x_hist': xhist_i2
        },
        {
            'method': 'FISTA+restart',
            'regularizer': name,
            'time_s': t_fista2,
            'step_norms': step_f2,
            'ratio_vals': ratio_f2,
            'x_hist': xhist_f2
        }
    ]

# Plot step-norms for ISTA+stop
plt.figure(figsize=(6,4))
for run in adv_results:
    if run['method']=='ISTA+stop':
        plt.semilogy(run['step_norms'], label=run['regularizer'])
plt.title('ISTA with Stopping: Step Norms vs Iteration')
plt.xlabel('Iteration number $k$')
plt.ylabel(r'Step norm $\|x^{k+1}-x^k\|$')
plt.grid(True, which='both', ls='--', lw=0.5)
plt.legend(title='Regularizer')
plt.tight_layout(); plt.show()

# Plot step-norms for FISTA+restart
plt.figure(figsize=(6,4))
for run in adv_results:
    if run['method']=='FISTA+restart':
        plt.semilogy(run['step_norms'], label=run['regularizer'])
plt.title('FISTA with Adaptive Restart: Step Norms vs Iteration')
plt.xlabel('Iteration number $k$')
plt.ylabel(r'Step norm $\|x^{k+1}-x^k\|$')
plt.grid(True, which='both', ls='--', lw=0.5)
plt.legend(title='Regularizer')
plt.tight_layout(); plt.show()

# Plot adaptive-restart ratio
plt.figure(figsize=(6,4))
for run in adv_results:
    if run['method']=='FISTA+restart':
        plt.semilogy(run['ratio_vals'], label=run['regularizer'])
plt.axhline(1.0, linestyle='--', linewidth=1, color='gray')
plt.title('FISTA Adaptive Restart: Ratio $\|Δx_k\|/\|Δx_{k-1}\|$')
plt.xlabel('Iteration number $k$')
plt.ylabel(r'Ratio $\|x^{k+1}-x^k\|/ \|x^k-x^{k-1}\|$')
plt.grid(True, which='both', ls='--', lw=0.5)
plt.legend(title='Regularizer')
plt.tight_layout(); plt.show()

Ridge Regression via L-BFGS

In [ ]:
# Configure and run L-BFGS solver for Ridge
ridge_solver = LBFGSSolver(reg_type='ridge', alpha1=0.0, alpha2=1.0, max_iter=500, tol=1e-6)
# Measure runtime
tr0 = time.perf_counter()
ridge_solver.fit(A, b)
tridge = time.perf_counter() - tr0

# Plot convergence envelope of L-BFGS for Ridge
ks_r = np.arange(1, len(ridge_solver.history_) + 1)
plt.figure(figsize=(6,4))
plt.loglog(
    ks_r,
    np.minimum.accumulate(np.array(ridge_solver.history_) - ridge_solver.final_obj_),
    '-', label=f"L-BFGS Ridge (α₁=0.0, α₂=1.0)"
)
plt.title("Convergence: Ridge Regression via L-BFGS (α₁=0.0, α₂=1.0)")
plt.xlabel('Iteration number $k$')
plt.ylabel('Suboptimality: $f(x) - f^*$')
plt.grid(True, which='both', ls='--', lw=0.5)
plt.legend(title='Method')
plt.tight_layout()
plt.show()

# Append summary to records
ridge_summary = {
    'method': 'L-BFGS',
    'regularizer': 'ridge',
    'alpha1': 0.0,
    'alpha2': 1.0,
    'iterations': len(ridge_solver.history_),
    'runtime_s': tridge
}

FISTA vs FISTA-Delta Comparison

In [ ]:
# Compare standard FISTA and FISTA-Delta for Lasso and ElasticNet cases
variants = []
delta = 3.0
for reg in basic_regs:
    name, a1, a2 = reg['name'], reg['alpha1'], reg['alpha2']
    if name not in ('lasso','elasticnet') or a1 <= 0.0:
        continue
    # Standard FISTA
    t0 = time.perf_counter()
    x_std, obj_std = fista(A, b, name, a1, a2, max_iter=500)
    t_std = time.perf_counter() - t0
    # FISTA-Delta
    t0 = time.perf_counter()
    x_del, obj_del = fista_delta(A, b, name, a1, a2, delta=delta, max_iter=500)
    t_del = time.perf_counter() - t0
    # Estimate best f*
    f_star = min(obj_std[-1], obj_del[-1])
    # Plot convergence envelopes
    ks_std = np.arange(1, len(obj_std)+1)
    ks_del = np.arange(1, len(obj_del)+1)
    env_std = np.minimum.accumulate(np.array(obj_std) - f_star)
    env_del = np.minimum.accumulate(np.array(obj_del) - f_star)
    plt.figure(figsize=(6,4))
    plt.loglog(ks_std, env_std, '-', label=f"FISTA standard (α₁={a1}, α₂={a2})")
    plt.loglog(ks_del, env_del, '-', label=f"FISTA-Δ δ={delta} (α₁={a1}, α₂={a2})")
    plt.title(f"FISTA vs FISTA-Δ Convergence: {name.title()} (α₁={a1}, α₂={a2})")
    plt.xlabel('Iteration number $k$')
    plt.ylabel('Suboptimality: $f(x) - f^*$')
    plt.grid(True, which='both', ls='--', lw=0.5)
    plt.legend(title='Method')
    plt.tight_layout()
    plt.show()
    # Record summary
    variants += [
        {'method':'FISTA','delta':None,'regularizer':name,'alpha1':a1,'alpha2':a2,'iterations':len(obj_std),'runtime_s':t_std},
        {'method':'FISTA-DELTA','delta':delta,'regularizer':name,'alpha1':a1,'alpha2':a2,'iterations':len(obj_del),'runtime_s':t_del}
    ]
# Aggregate variant results into DataFrame